In [0]:
dbutils.widgets.text("sql_group","TESTCASEA")

In [0]:
sql_group = dbutils.widgets.get("sql_group")
print(sql_group)

### Get all the rows for a specific sql_group and belonging to sqlpool target type

In [0]:
config_df = spark.sql(f"SELECT * FROM v_sql_load_config WHERE sql_group = '{sql_group}' AND sql_type = 'sqlpool'")
display(config_df)

In [0]:
import time
import json

notebook_name = "Child_notebook"
tenantId = "dac94bb8-0c1d-4d66-8a58-e02c2406c486"

def write_to_sql(param):
  src_table = param["src_table"]
  target_table = param["trgt_table"]
  sql_group = param["sql_group"]
  db_name = param["db_name"]
  mode = param["mode"]
  hostname = param["hostname"]
  key_scope = param["key_scope"]
  cred_type = param["cred_type"]
  credentials = param["credentials"]

  print(f"Table {target_table} started")

  result = dbutils.notebook.run(f"./{notebook_name}", 1200, {"src_table": src_table, "target_table":target_table, "sql_group":sql_group, "db_name": db_name, "mode":mode,"hostname": hostname, "key_scope": key_scope, "cred_type": cred_type, "userid": credentials['userid'], "password": credentials['password'], "tenantId": tenantId})
  return json.loads(result)  

## Call the SQL writing function in multi threaded format

In [0]:
import concurrent.futures
start_time = time.perf_counter()
worker_count = 8

cols_args = config_df.toPandas().to_dict('records') 

with concurrent.futures.ThreadPoolExecutor(worker_count) as executor:
    futures = {executor.submit(write_to_sql, param):  param for param in cols_args}
        
final_result = []
for f in concurrent.futures.as_completed(futures):
    final_result.append(f.result())
print(f"Final json result : {final_result}")
end_time = time.perf_counter()
total_time = end_time - start_time
print(f"Total time : {total_time}")